## Implementing Chris Doenlen's 'Bot Or Not' Python Module

Everything from `twitter_funcs.py` was cloned from Chris' [repository](https://github.com/scrapfishies/twitter-bot-detection).

I will use this to label each user as 'bot' (boolean 1/0).

In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from twitter_funcs import *

# Imports
import os
import numpy as np
import pandas as pd

import pickle
import tweepy

from datetime import datetime
from secrets import api_secret_key, api_key, bearer_token
import re
import time
import csv

In [2]:
data = pd.read_pickle("pickle/n2_tokenized_eff.pick")

In [3]:
data.head(3)

,id,date,time,user_id,username,hashtags,trump,biden,original,tweet,num_tokens
181142,1323379284434669568,2020-11-02,21:39:43,2820503362,artistacriseida,[],False,True,All these articles showing that Biden is in th...,article showing lead ignore govote map showing...,16
0,1323414585995526144,2020-11-02,23:59:59,1312487180258820096,annapieters17,[],False,True,@FoxNews Lady Gaga’s a nobody. Can’t figure ou...,foxnews lady nobody figure life nobody help ga...,10
4,1323414585232293888,2020-11-02,23:59:59,2335763630,kylechwatt,[],False,True,@The_Grupp “It is purely a fortuity that this ...,purely fortuity mass casualty event history ch...,8


I'm getting a rate limit error. according to twitter site, I can lookup 300 users per 15 minutes. Let's try that.

In [4]:
def chunks(user_ids, n):
    """Yield successive n-sized chunks from user_ids (iterable)."""
    lst = list(user_ids)
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
    


Here's where we'll implement Chris Doenlen's 'Bot or Not' model.

In [5]:
with open("bot_model.pick", "rb") as read_file:
    xgb_model = pickle.load(read_file)

In [6]:
# time.sleep(60*15)

In [11]:
# verifieds = {}
# bot_probas = {}

# all_users = list(data.user_id.unique())
# print(f"Total number of users to scrape: {len(all_users)}")
# exist = pd.read_csv("../data/user_stats.csv")
# exist_users = list(exist.user_id.unique())
# print(f"Number of users already scraped: {len(exist_users)}")

# user_ids = []
# for user in exist_users:
#     if user in all_users:
#         continue
#     else:
#         user_ids.append(user)
    
# print(f"Preparing to identify bots for {len(user_ids)} users...")

# user_id_chunks = list(chunks(user_ids, n=300))

# # now get stats for new users

# csv_file = open("../data/user_stats.csv", "a")
# csv_writer = csv.writer(csv_file)
# csv_writer.writerow(["user_id", "bot_proba", "verified"])
# for chunk in user_id_chunks:
#     print(f"Preparing chunk. Num users: {len(chunk)}")
#     for user_id in chunk:
#         print(f"Preparing user '{user_id}'")
        
        
#         auth = tweepy.OAuthHandler(api_key, api_secret_key)
#         api = tweepy.API(auth)
        
#         try: # Gather features for bot/not bot model
#             # Get user information from screen name
#             user = api.get_user(user_id)

#             # account features to return for predicton
#             account_age_days = (datetime.now() - user.created_at).days
#             verified = user.verified # will also use this in our data
#             geo_enabled = user.geo_enabled
#             default_profile = user.default_profile
#             default_profile_image = user.default_profile_image
#             favourites_count = user.favourites_count
#             followers_count = user.followers_count
#             friends_count = user.friends_count
#             statuses_count = user.statuses_count
#             average_tweets_per_day = np.round(statuses_count / account_age_days, 3)

#             # manufactured features
#             hour_created = int(user.created_at.strftime("%H"))
#             network = np.round(np.log(1 + friends_count) * np.log(1 + followers_count), 3)
#             tweet_to_followers = np.round(
#                 np.log(1 + statuses_count) * np.log(1 + followers_count), 3
#             )
#             follower_acq_rate = np.round(
#                 np.log(1 + (followers_count / account_age_days)), 3
#             )
#             friends_acq_rate = np.round(np.log(1 + (friends_count / account_age_days)), 3)

#             # organizing list to be returned
#             account_features = [
#                 verified, hour_created,geo_enabled,default_profile,default_profile_image,favourites_count,
#                 followers_count,friends_count,statuses_count,average_tweets_per_day,network,tweet_to_followers,
#                 follower_acq_rate,friends_acq_rate]

#             if account_features == np.nan:
#                 proba = np.nan
#                 verified = np.nan
#                 csv_writer.writerow([user_id, proba, verified])
#                 continue

#             else:
#                 user_m = np.matrix(account_features)
#                 proba = np.round(xgb_model.predict_proba(user_m)[:, 1][0] * 100, 2)
#                 verified = account_features[0]
#                 csv_writer.writerow([user_id, proba, verified])

#         except:
#             print(f'error encountered, skipping user {user_id}')
#             proba = np.nan
#             verified = np.nan
        
#             csv_writer.writerow([user_id, proba, verified])
#     print("Chunk complete. Waiting 15 minutes.")
#     time.sleep(15*60+1)

# csv_file.close()

In [ ]:
# one-off copy
time.sleep(120)
verifieds = {}
bot_probas = {}

user_ids = list(pd.read_pickle("pickle/grab_users.pick"))
    

user_id_chunks = list(chunks(user_ids, n=300))

# now get stats for new users

csv_file = open("../data/user_stats.csv", "a")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["user_id", "bot_proba", "verified"])
for chunk in user_id_chunks:
    print(f"Preparing chunk. Num users: {len(chunk)}")
    for user_id in chunk:
        print(f"Preparing user '{user_id}'")
        
        
        auth = tweepy.OAuthHandler(api_key, api_secret_key)
        api = tweepy.API(auth)
        
        try: # Gather features for bot/not bot model
            # Get user information from screen name
            user = api.get_user(user_id)

            # account features to return for predicton
            account_age_days = (datetime.now() - user.created_at).days
            verified = user.verified # will also use this in our data
            geo_enabled = user.geo_enabled
            default_profile = user.default_profile
            default_profile_image = user.default_profile_image
            favourites_count = user.favourites_count
            followers_count = user.followers_count
            friends_count = user.friends_count
            statuses_count = user.statuses_count
            average_tweets_per_day = np.round(statuses_count / account_age_days, 3)

            # manufactured features
            hour_created = int(user.created_at.strftime("%H"))
            network = np.round(np.log(1 + friends_count) * np.log(1 + followers_count), 3)
            tweet_to_followers = np.round(
                np.log(1 + statuses_count) * np.log(1 + followers_count), 3
            )
            follower_acq_rate = np.round(
                np.log(1 + (followers_count / account_age_days)), 3
            )
            friends_acq_rate = np.round(np.log(1 + (friends_count / account_age_days)), 3)

            # organizing list to be returned
            account_features = [
                verified, hour_created,geo_enabled,default_profile,default_profile_image,favourites_count,
                followers_count,friends_count,statuses_count,average_tweets_per_day,network,tweet_to_followers,
                follower_acq_rate,friends_acq_rate]

            if account_features == np.nan:
                proba = np.nan
                verified = np.nan
                csv_writer.writerow([user_id, proba, verified])
                continue

            else:
                user_m = np.matrix(account_features)
                proba = np.round(xgb_model.predict_proba(user_m)[:, 1][0] * 100, 2)
                verified = account_features[0]
                csv_writer.writerow([user_id, proba, verified])

        except:
            print(f'error encountered, skipping user {user_id}')
            proba = np.nan
            verified = np.nan
        
            csv_writer.writerow([user_id, proba, verified])
    print("Chunk complete. Waiting 15 minutes.")
    time.sleep(15*60+1)

csv_file.close()

Preparing chunk. Num users: 300
Preparing user '1143289465387081732'
Preparing user '782603330'
Preparing user '777679662310907904'
Preparing user '1408230732'
Preparing user '20406676'
Preparing user '1304172941391863817'
Preparing user '51827732'
Preparing user '3937940121'
Preparing user '8500962'
Preparing user '570097148'
Preparing user '104246758'
Preparing user '1043423847708590081'
Preparing user '31571443'
Preparing user '547892272'
Preparing user '160364793'
Preparing user '1251323699451564035'
Preparing user '20824974'
Preparing user '51183401'
Preparing user '1649886427'
Preparing user '15145034'
Preparing user '112557979'
Preparing user '3304837284'
Preparing user '832841680135081984'
Preparing user '1241019363525677057'
Preparing user '1255552876073680903'
Preparing user '1287515682641412096'
Preparing user '817761228688916480'
Preparing user '1239930170800107521'
Preparing user '283350568'
Preparing user '850046670'
Preparing user '728930499029172224'
Preparing user '444

Preparing user '207656440'
Preparing user '912135002782969857'
Preparing user '212795909'
Preparing user '2362868665'
Preparing user '72592074'
Preparing user '585764087'
Preparing user '1072212454686515209'
Preparing user '1102655017042038785'
Preparing user '91808599'
Preparing user '1176325494259798017'
Preparing user '1289930821357445132'
Preparing user '34283650'
Preparing user '67035595'
Preparing user '1375297094'
Preparing user '1230089817418895360'
Preparing user '993215081256181760'
Preparing user '26965771'
Preparing user '1220666675072204801'
Preparing user '41964897'
Preparing user '219174241'
Preparing user '1016066208'
Preparing user '529718790'
Preparing user '1041056512582533121'
Preparing user '124275780'
Preparing user '1180983441044770817'
Preparing user '3167620869'
Preparing user '1234187035'
Preparing user '1309975396943880192'
Preparing user '17773224'
Preparing user '94823672'
Preparing user '26928006'
Preparing user '802581350846844928'
Preparing user '2115967

Preparing user '31285825'
Preparing user '3396389877'
Preparing user '1204945320079175681'
Preparing user '1291911212163358720'
Preparing user '465897557'
Preparing user '825310374316634112'
Preparing user '2966660663'
Preparing user '565151401'
Preparing user '1578369842'
Preparing user '3343545432'
Preparing user '2360295679'
Preparing user '706840182788636672'
Preparing user '4754857394'
Preparing user '21549689'
Preparing user '1132095449358323715'
Preparing user '18286742'
Preparing user '1188920021625401344'
Preparing user '3834650237'
Preparing user '330361451'
Preparing user '923200791904702464'
Preparing user '1229952949691940864'
Preparing user '2335805793'
Preparing user '176514190'
Preparing user '3177941678'
Preparing user '897872740593651712'
Preparing user '37426831'
Preparing user '23645254'
Preparing user '300849011'
Preparing user '1215484675701035008'
Preparing user '736277975226683394'
Preparing user '211747133'
Preparing user '2739941258'
Preparing user '968926454'

Preparing user '1122303946385547264'
Preparing user '1307520297235439618'
Preparing user '281443650'
Preparing user '971925509285335042'
Preparing user '1363755902'
Preparing user '1131264983432597504'
Preparing user '1100277946055516160'
Preparing user '836004766262034432'
Preparing user '1214625199'
Preparing user '1051877297605799937'
Preparing user '370938830'
Preparing user '3018098473'
Preparing user '108987695'
Preparing user '1258523139606147072'
Preparing user '1127523210277396482'
Preparing user '74820061'
Preparing user '427901036'
Preparing user '1450670106'
Preparing user '172469352'
Preparing user '269076835'
Preparing user '224274349'
Preparing user '4458421461'
Preparing user '250944832'
Preparing user '49961271'
Preparing user '1276017627358347264'
Preparing user '78865371'
Preparing user '1254174805072457734'
Preparing user '898651031508324353'
Preparing user '18449140'
Preparing user '296838793'
Preparing user '1267918590432477184'
Preparing user '44981090'
Preparing

Preparing user '1087298936166076417'
Preparing user '1150911242426552322'
Preparing user '273135082'
Preparing user '363307821'
Preparing user '3120948660'
Preparing user '36046951'
Preparing user '796933268499267584'
Preparing user '1063105875177586689'
Preparing user '227552105'
Preparing user '2722253653'
Preparing user '1208848039437205504'
Preparing user '1589924701'
Preparing user '6634632'
Preparing user '1153003898887692288'
Preparing user '2901851217'
Preparing user '17723942'
Preparing user '1269067340802666504'
Preparing user '3702282797'
Preparing user '765355137263173632'
Preparing user '516345781'
Preparing user '2591789040'
Preparing user '795267601957912576'
Preparing user '1911873032'
Preparing user '1275524614723002368'
Preparing user '44806272'
Preparing user '989035910460334081'
Preparing user '319931473'
Preparing user '1242501675299921921'
Preparing user '824460088744239106'
Preparing user '1088098164031262720'
Preparing user '2161737235'
Preparing user '277887693

...

Preparing user '1191934183'
Preparing user '754115933963030528'
Preparing user '917122387107135488'
Preparing user '188867689'
Preparing user '901976017203494913'
Preparing user '21111787'
Preparing user '2932638062'
Preparing user '1684930806'
Preparing user '1258882628503187456'
Preparing user '1268487678527369216'
Preparing user '1277537041337561089'
Preparing user '51343172'
Preparing user '2177258582'
Preparing user '1196068073851232256'
Preparing user '24193289'
Preparing user '2737530630'
Preparing user '4739490390'
Preparing user '22105251'
Preparing user '1247667125562404864'
Preparing user '489692221'
Preparing user '2183681736'
Preparing user '1214656991664070657'
Preparing user '1034675221452578816'
Preparing user '752901953571565570'
Preparing user '14911096'
Preparing user '17312657'
Preparing user '1286787538062118917'
Preparing user '381183215'
Preparing user '900704527787196417'
Preparing user '1654016677'
Preparing user '307264661'
Preparing user '382528490'
Preparing

Preparing user '20684040'
Preparing user '1206686299484295169'
Preparing user '1710316669'
Preparing user '31839947'
Preparing user '1173813157'
Preparing user '182037735'
Preparing user '20536600'
Preparing user '994804499695972354'
Preparing user '960741564803887105'
Preparing user '1296835668073689089'
Preparing user '1135701193'
Preparing user '1353039818'
Preparing user '1329369103'
Preparing user '829515153771208706'
Preparing user '399126583'
Preparing user '1160067083197132800'
Preparing user '89076502'
Preparing user '1284765619821268992'
Preparing user '1344920712'
Preparing user '557018736'
Preparing user '1225855095994298368'
Preparing user '875820021137670150'
Preparing user '981727945390604288'
Preparing user '980317540881698816'
Preparing user '1269625604305010694'
Preparing user '247124972'
Preparing user '102441591'
Preparing user '1221294634514690049'
Preparing user '223704779'
Preparing user '14933405'
Preparing user '1234151516950253568'
Preparing user '21385020'
Pr

Preparing user '28044313'
Preparing user '3328688975'
Preparing user '1017096381863882752'
Preparing user '1030655401585516545'
Preparing user '339687246'
Preparing user '14885073'
Preparing user '47063142'
Preparing user '1260071551543713792'
Preparing user '1293137707284287489'
Preparing user '1304923510750556160'
Preparing user '2735404548'
Preparing user '374901847'
Preparing user '1251306238882639872'
Preparing user '85505639'
Preparing user '774219673'
Preparing user '31488410'
Preparing user '1031168813746802689'
Preparing user '822922134364139520'
Preparing user '946760044913266688'
Preparing user '1278141990735974405'
Preparing user '486719429'
Preparing user '2404263109'
Preparing user '42299623'
Preparing user '1082582643529732097'
Preparing user '307410239'
Preparing user '71836637'
Preparing user '1094648156309065728'
Preparing user '3223426134'
Preparing user '2923333369'
Preparing user '1699684640'
Preparing user '55846083'
Preparing user '2520282677'
Preparing user '588

Preparing user '1597349443'
Preparing user '1239265166749024256'
Preparing user '1548453757'
Preparing user '824299755278843905'
Preparing user '366403573'
Preparing user '2835436748'
Preparing user '899982851466289152'
Preparing user '783223518'
Preparing user '1282566923478159361'
Preparing user '164499168'
Preparing user '1302655091644731393'
Preparing user '444907867'
Preparing user '950380857545646080'
Preparing user '83013225'
Preparing user '436344781'
Preparing user '3334819978'
Preparing user '1204827888492711938'
Preparing user '590948423'
Preparing user '1247198545'
Preparing user '124327235'
Preparing user '46693313'
Preparing user '21865699'
Preparing user '291307906'
Preparing user '1047503025693253633'
Preparing user '3241131838'
Preparing user '1311342555137798145'
Preparing user '148125233'
Preparing user '3316713321'
Preparing user '1150196778211454976'
Preparing user '920737176303915012'
Preparing user '2360187944'
Preparing user '343713611'
Preparing user '757739089

Preparing user '2180749375'
Preparing user '230904554'
Preparing user '301265349'
Preparing user '108148552'
Preparing user '109621134'
Preparing user '824243179281117185'
Preparing user '704083357840379906'
Preparing user '1005393387397308416'
Preparing user '757013945198612480'
Preparing user '603116312'
Preparing user '1164004682211180544'
Preparing user '221490171'
Preparing user '1898343469'
Preparing user '285287627'
Preparing user '1285890474893549569'
Preparing user '1312088599551119361'
Preparing user '1198926764061405185'
Preparing user '341221840'
Preparing user '74286598'
Preparing user '1011607827482152961'
Preparing user '1254187707837960193'
Preparing user '121651277'
Preparing user '828248641206890496'
Preparing user '31053294'
Preparing user '1278025203855982592'
Preparing user '222991974'
Preparing user '942190051122925568'
Preparing user '714070711468367872'
Preparing user '3230917482'
Preparing user '121849626'
Preparing user '1050685802273886208'
Preparing user '19

In [ ]:
bot_probas

In [ ]:
verifieds

In [ ]:
data['verified'] = data['user_id'].map(verifieds)
data['bot_proba'] = data['user_id'].map(bot_probas)
